## 基于高德地图地理编码与逆地理编码API的数据地点层次化处理
API文档：https://lbs.amap.com/api/webservice/guide/api/georegeo

In [1]:
import pandas as pd
import urllib
import numpy as np

In [2]:
path = 'C:/Users/new/Desktop/疫情演化系统/数据/train_township.csv'
data = pd.read_csv(path)

In [3]:
data.head()

,日期,镇区,经停病例数,患者数量
0,2021/12/23,东马圈镇,0,0
1,2021/12/24,东马圈镇,0,0
2,2021/12/25,东马圈镇,0,0
3,2021/12/26,东马圈镇,0,0
4,2021/12/27,东马圈镇,0,0


In [4]:
data['完整地址'] = ''
data['所在区'] = ''
data['街道'] = ''
data['township'] = ''

In [5]:
data.head()

,日期,镇区,经停病例数,患者数量,完整地址,所在区,街道,township
0,2021/12/23,东马圈镇,0,0,,,,
1,2021/12/24,东马圈镇,0,0,,,,
2,2021/12/25,东马圈镇,0,0,,,,
3,2021/12/26,东马圈镇,0,0,,,,
4,2021/12/27,东马圈镇,0,0,,,,


In [4]:
API_key = "947c7f60f731b54bd6474ce18d5d894b"  # 使用高德地图地理编码API所需的key,这里我申请的是个人key,每日只能查询5000条数据
city = "天津"  # 默认对天津的地点进行地理编码和逆编码
city = urllib.parse.quote(city)  # 这里要注意,按照标准,URL只允许一部分ASCII字符,其他字符（如汉字）是不符合标准的，此时就要进行编码,下方的address同样需要

In [5]:
address = ''
for index,row in data.iterrows():
    if address != row['镇区']:
        address = row['镇区']
        if address is not np.NaN:
            address = urllib.parse.quote(address)
            geo_url = "https://restapi.amap.com/v3/geocode/geo?key={}&address={}&city={}".format(API_key,address,city)

            req = urllib.request.Request(geo_url)  # 将url对象转化为request对象
            ret = urllib.request.urlopen(req)  # 发送请求，打开url，也可以直接传入url
            result = ret.read().decode('utf-8')

            result = eval(result)
            if result['status'] == '1':  # 有时候API会有一些没法返回正常的查询结果，比如乐齐国际影城，不知道为啥返回不了正常结果，status=0
                location = result['geocodes'][0]['location']
                longitude, latitude = eval(location)  # 地点对应经纬度

                regeo_url = 'https://restapi.amap.com/v3/geocode/regeo?key={}&location={}&extensions=base'.format(API_key,location)  # extensions 参数取值为 all 时会返回基本地址信息、附近 POI 内容、道路信息以及道路交叉口信息
                req = urllib.request.Request(regeo_url)  # 将url对象转化为request对象
                ret = urllib.request.urlopen(req)  # 发送请求，打开url，也可以直接传入url
                result = ret.read().decode('utf-8')

                result = eval(result)
                if result['status'] == '1':
                    formatted_address = result['regeocode']['formatted_address']  # 完整地址
                    district = result['regeocode']['addressComponent']['district']  # 所在区
                    street = result['regeocode']['addressComponent']['streetNumber']['street']  # 所在街道
                    township = result['regeocode']['addressComponent']['township']  # 暂不了解此词条的含义
                    # formatted_address
                    # district
                    # street
                    # township
    #                 if formatted_address:  # 主要是防止有些地址返回的是空集
    #                     data.loc[index,'完整地址'] = formatted_address
    #                 if district:
    #                     data.loc[index,'所在区'] = district
    #                 if street:
    #                     data.loc[index,'街道'] = street
    #                 if township:
    #                     data.loc[index,'township'] = township
                    if longitude:
                        data.loc[index,'经度'] = longitude
                    if latitude:
                        data.loc[index,'纬度'] = latitude

KeyboardInterrupt: 

In [6]:
address

'%E4%B8%9C%E4%B8%BD%E5%8C%BA%E5%BC%80%E5%8F%91%E5%8C%BA'

In [7]:
data.head()

,日期,镇区,经停病例数,患者数量,经度,纬度
0,2021/12/23,东马圈镇,0,0,116.841991,39.44891
1,2021/12/24,东马圈镇,0,0,116.841991,39.44891
2,2021/12/25,东马圈镇,0,0,116.841991,39.44891
3,2021/12/26,东马圈镇,0,0,116.841991,39.44891
4,2021/12/27,东马圈镇,0,0,116.841991,39.44891


In [8]:
outputpath='C:/Users/new/Desktop/Geocoding.csv'
data.to_csv(outputpath,sep=',',index=False,header=True,encoding='utf_8_sig')  # 保存结果